In [ ]:
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE TypeFamilies #-}
{-# LANGUAGE AllowAmbiguousTypes #-}
{-# LANGUAGE TypeApplications #-}
{-# LANGUAGE ScopedTypeVariables #-}

import Control.Monad.ST
import Control.Monad.Primitive
import Data.Vector.Generic.Base
import qualified Data.Vector as V
import qualified Data.Vector.Generic.Mutable as MV
import qualified Data.Vector.Generic as MG
import System.Random.MWC

In [ ]:
repeatSwap :: (PrimMonad m, MV.MVector v a) => v (PrimState m) a -> Gen (PrimState m) -> Int ->  m (v (PrimState m) a)
repeatSwap v g 0 = return v
repeatSwap v g n = do
    let n' = n - 1
    i <- uniformR (0, MV.basicLength v) g
    MV.swap v i n'
    repeatSwap v g n'

In [ ]:
body :: forall m a. (PrimMonad m) => [a] -> Gen (PrimState m) -> m [a]
body xs g = do
    v <- MG.unsafeThaw $ (MG.fromList @V.Vector) xs
    repeatSwap v g $ MV.basicLength v
    MG.toList <$> MG.unsafeFreeze v

In [ ]:
shuffleVector :: [a] -> IO [a]
shuffleVector xs = withSystemRandom . asGenIO $ body xs

In [ ]:
print <$> shuffleVector [1, 2, 3, 4, 5, 6]

In [ ]:
:info mkMutable

In [ ]:
shuffleList :: [a] -> IO [a]
shuffleList [a] = return [a]
shuffleList xs =  withSystemRandom . asGenIO $ \g -> do
    v <- MG.unsafeThaw $ (MG.fromList @V.Vector) xs
    repeatSwap v g n
    MG.toList <$> MG.unsafeFreeze v
        where
            n = length xs
            repeatSwap v g 0 = return v
            repeatSwap v g i = do
                let i' = i - 1
                j <- uniformR (0, n) g
                MV.swap v j i'
                repeatSwap v g i'

In [ ]:
:info shuffleList

In [ ]:
print <$> shuffleList @(ST Vector Integer) [1..10]